# Chapter 6 — Diagonalization of Matrices (Colab Companion)

This notebook provides **Python helpers and demos** for diagonalization topics:

- **Similarity and change of basis:** \(A \sim D\) via \(A = P D P^{-1}\)
- **Diagonalizability criteria:** distinct eigenvalues; algebraic vs geometric multiplicities
- **Diagonalization over \(\mathbb{C}\)** vs over \(\mathbb{R}\) (complex eigenvalues)
- **Orthogonal diagonalization of real symmetric matrices** (spectral theorem)
- Using diagonalization to compute **powers** \(A^k\), **polynomials** \(p(A)\), and **matrix exponentials** \(e^{tA}\)
- Recognizing **defective** (non-diagonalizable) matrices

All explanatory comments are in **English** and we rely on **SymPy** for exact linear algebra.

In [ ]:
# Setup (SymPy). In Colab SymPy is usually available.
# If needed, you can uncomment the next line to install/upgrade:
# !pip -q install sympy

import sympy as sp

sp.init_printing(use_unicode=True)

def to_sym(A):
    """Coerce input to a SymPy Matrix."""
    if isinstance(A, sp.MatrixBase):
        return A
    return sp.Matrix(A)

## 1) Similarity and change of basis

`A` and `B` are **similar** if `B = P^{-1} A P` for some invertible `P`. If `B` is diagonal, we say **A is diagonalizable**.

In [ ]:
def similarity_transform(A, P):
    """Return B = P^{-1} A P (requires invertible P)."""
    A, P = to_sym(A), to_sym(P)
    if P.det() == 0:
        raise ValueError("P must be invertible for a similarity transform.")
    return P.inv() * A * P

def verify_diagonalization(A, P, D):
    """Check that A == P*D*P^{-1} and D is diagonal."""
    A, P, D = to_sym(A), to_sym(P), to_sym(D)
    is_diag = D == sp.diag(*D.diagonal())  # quick check
    return bool((P*D*P.inv() - A) == sp.zeros(*A.shape) and is_diag)

## 2) Diagonalizability criteria and construction

A sufficient condition: **distinct eigenvalues** \(\Rightarrow\) diagonalizable (over \(\mathbb{C}\)).  
In general, **A is diagonalizable iff the sum of geometric multiplicities equals n**.

In [ ]:
def eigen_multiplicities(A):
    """Return a list of dicts with eigenvalue, algebraic multiplicity, geometric multiplicity."""
    A = to_sym(A)
    n = A.shape[0]
    data = []
    # SymPy's eigenvals() -> {eigenvalue: algebraic multiplicity}
    for lam, am in A.eigenvals().items():
        geom_mult = len((A - lam*sp.eye(n)).nullspace())
        data.append({"eigenvalue": lam, "alg_mult": int(am), "geom_mult": int(geom_mult)})
    return data

def has_distinct_eigenvalues(A):
    """True if all algebraic multiplicities are 1 (over algebraic closure)."""
    A = to_sym(A)
    return all(m == 1 for m in A.eigenvals().values())

def is_diagonalizable_over_C(A):
    """Diagonalizable over C iff sum of geometric multiplicities equals n."""
    A = to_sym(A)
    n = A.shape[0]
    mults = eigen_multiplicities(A)
    return sum(item["geom_mult"] for item in mults) == n

def is_diagonalizable_over_R(A):
    """Attempt a real diagonalization check: all eigenvalues real and sum geom mult == n."""
    A = to_sym(A)
    n = A.shape[0]
    evals = A.eigenvals()
    # All eigenvalues must be real
    for lam in evals.keys():
        if not lam.is_real:
            return False
    # Geometric multiplicities with real eigenvectors
    total_geom = 0
    for lam, am in evals.items():
        geom_mult = len((A - lam*sp.eye(n)).nullspace())
        total_geom += geom_mult
    return total_geom == n

def diagonalize_over_C(A):
    """Return (P, D) such that A = P*D*P^{-1}, or (None, None) if not diagonalizable over C."""
    A = to_sym(A)
    n = A.shape[0]
    cols = []
    vals = []
    for lam, am in A.eigenvals().items():
        basis = (A - lam*sp.eye(n)).nullspace()
        for v in basis:
            cols.append(v)
            vals.append(lam)
    if len(cols) != n:
        return None, None
    P = sp.Matrix.hstack(*cols)
    if P.det() == 0:
        return None, None
    D = sp.diag(*vals)
    return P, D

## 3) Spectral theorem (real symmetric case)

If `A` is **real symmetric** then there exists an **orthogonal** matrix `Q` such that  
`A = Q Λ Q^T` with `Λ` diagonal and **real** eigenvalues.

In [ ]:
def is_real_symmetric(A):
    A = to_sym(A)
    return bool(A.equals(A.T) and all(entry.is_real for entry in A))

def spectral_decomposition_symmetric(A):
    """Return (Q, Λ) with Q orthogonal and Λ diagonal for a real symmetric A."""
    A = to_sym(A)
    if not is_real_symmetric(A):
        raise ValueError("A must be real symmetric for an orthogonal diagonalization.")
    # SymPy: use eigenvects to collect an orthonormal eigenbasis via Gram-Schmidt on each eigenspace
    n = A.shape[0]
    eigspaces = []
    vals = []
    for lam, mult in A.eigenvals().items():
        basis = (A - lam*sp.eye(n)).nullspace()
        # Gram-Schmidt to orthonormalize basis w.r.t. standard inner product
        ortho = sp.GramSchmidt(basis, orthonormal=True)
        eigspaces.extend(ortho)
        vals.extend([lam] * len(ortho))
    Q = sp.Matrix.hstack(*eigspaces)
    Λ = sp.diag(*vals)
    # Normalize to ensure Q^T Q = I
    return Q, Λ

## 4) Using diagonalization: powers, polynomials, exponentials

If `A = P D P^{-1}`, then:
- `A^k = P D^k P^{-1}`
- `p(A) = P p(D) P^{-1}` (apply `p` to the diagonal entries)
- `e^{tA} = P e^{tD} P^{-1}`

In [ ]:
def power_via_diagonalization(A, k):
    A = to_sym(A)
    P, D = diagonalize_over_C(A)
    if P is None:
        # Fallback to direct power
        return A**k
    Dk = sp.diag(*[d**k for d in D.diagonal()])
    return P * Dk * P.inv()

def mat_poly_horner(A, coeffs):
    """Evaluate p(A) with Horner's method; coeffs in ascending powers [c0, c1, ..., ck]."""
    A = to_sym(A)
    n = A.shape[0]
    P = sp.zeros(n)
    I = sp.eye(n)
    ck = coeffs[-1]
    P = ck * I
    for c in reversed(coeffs[:-1]):
        P = A*P + c*I
    return P

def polynomial_via_diagonalization(A, coeffs):
    """Compute p(A) via diagonalization if possible; else Horner.
    coeffs: [c0, c1, ..., ck] for p(λ) = c0 + c1 λ + ... + ck λ^k
    """
    A = to_sym(A)
    P, D = diagonalize_over_C(A)
    if P is None:
        return mat_poly_horner(A, coeffs)
    # Apply p to each diagonal entry
    diag_vals = []
    for lam in D.diagonal():
        val = 0
        pow_lam = 1
        for c in coeffs:
            val += c * pow_lam
            pow_lam *= lam
        diag_vals.append(sp.simplify(val))
    pD = sp.diag(*diag_vals)
    return P * pD * P.inv()

def exp_via_diagonalization(A, t=1):
    A = to_sym(A)
    P, D = diagonalize_over_C(A)
    if P is None:
        # Fallback to SymPy's matrix exponential
        return (t*A).exp()
    exp_diag = sp.diag(*[sp.exp(t*lam) for lam in D.diagonal()])
    return P * exp_diag * P.inv()

---

## 5) Quick demos

In [ ]:
# Demo A: Distinct eigenvalues => diagonalizable over C
A = sp.Matrix([[3, 1, 0],
               [0, 2, 0],
               [0, 0, -1]])
print("A ="); sp.pprint(A)
print("Distinct eigenvalues? ", has_distinct_eigenvalues(A))
print("Diagonalizable over C? ", is_diagonalizable_over_C(A))
P, D = diagonalize_over_C(A)
print("\nP ="); sp.pprint(P)
print("D ="); sp.pprint(D)
print("Verify A = P D P^{-1}? ->", verify_diagonalization(A, P, D))

In [ ]:
# Demo B: Real symmetric -> orthogonal diagonalization
S = sp.Matrix([[2, 1, 0],
               [1, 3, 0],
               [0, 0, 4]])
print("S real symmetric? ", is_real_symmetric(S))
Q, Lmbda = spectral_decomposition_symmetric(S)
print("\nQ orthogonal? ->", (Q.T*Q == sp.eye(S.shape[0])))
print("Λ ="); sp.pprint(Lmbda)
print("Check S == Q Λ Q^T ? ->", (Q*Lmbda*Q.T - S) == sp.zeros(*S.shape))

In [ ]:
# Demo C: Non-diagonalizable (Jordan block)
J = sp.Matrix([[1, 1],
               [0, 1]])
print("J diagonalizable over C? ", is_diagonalizable_over_C(J))
P, D = diagonalize_over_C(J)
print("P is None? ->", P is None)
print("J^5 via diagonalization fallback:"); sp.pprint(power_via_diagonalization(J, 5))

In [ ]:
# Demo D: Using diagonalization for powers, polynomials, exponentials
B = sp.Matrix([[0, 1],
               [1, 0]])  # eigenvalues ±1
print("B^10 ="); sp.pprint(power_via_diagonalization(B, 10))
# p(λ) = 2 + 3λ + λ^2
print("\np(B) = 2 I + 3 B + B^2:"); sp.pprint(polynomial_via_diagonalization(B, [2, 3, 1]))
print("\nexp(tB) with t=pi/3:"); sp.pprint(exp_via_diagonalization(B, sp.pi/3))

---

### How to use this notebook

1. Run the **Setup** cell.
2. Use **`eigen_multiplicities`**, **`has_distinct_eigenvalues`**, **`is_diagonalizable_over_C/R`** to test your matrix.
3. If diagonalizable, get **`(P, D)`** with **`diagonalize_over_C`** or **`spectral_decomposition_symmetric`** (real symmetric case).
4. Compute **`A^k`**, **`p(A)`**, or **`e^{tA}`** via the helper functions in Section 4.
5. Use **`similarity_transform`**/**`verify_diagonalization`** to sanity-check your results.